In [ ]:
import json
import os
from openai import OpenAI
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

##### referencing the json file

In [ ]:
with open("json file path") as f:
    idv_json = json.load(f)

##### the system prompt is written in that style using Chain of Thought strategy also One shot with an example given as part of problem statement

In [10]:
system_prompt = """
You are an intelligent Compliance assistant built to analyze a structured Identification Verification data.
Your role is to help end-users or support teams understand WHY their document verification passed or failed, and WHAT THEY SHOULD DO NEXT to fix any issues.

You are given a structured JSON input that may include the following fields:
- `document_type`: the type of ID document 
- `fields`: extracted data like name, date of birth, aadhaar number with confidence scores
- `face_match`: contains similarity scores comparing the user's selfie and ID photo
- `forgery_checks`: includes flags like glare, tampering, or template mismatches
- `compliance_checks`: includes results like age_check, masking_check, face_match_threshold
- `integrity_issues`: quality issues like cropping or low resolution

You have to:
1. Determine if the user passed or failed verification.
2. If failed, clearly explain all the specific reasons why.
   - Be accurate and structured.
   - Use the field names from the JSON but convert them into natural language.
3. Give helpful, specific, and actionable suggestions** for how the user can fix the problem.
   - Do not give generic advice. Tailor it to the actual issue.
   - Be polite and reassuring.
4. Respond in the same language as the `language_hint` field (default to English if not provided).
5. Do not output JSON or code — respond in natural language only.
6. Format your response into two sections:
   - Reasons for Failure
   - Recommended Actions

If everything is correct , respond that verification is successful and no action is needed.

understand the example below and try to resond in similar fashion:

for a json input such as this --
{
  "document_type": "AADHAAR_FRONT",
  "fields": {
    "name": { "value": "RAHUL SHARMA", "confidence": 0.67 },
    "dob": { "value": "2005-01-15", "confidence": 0.89 },
    "aadhaar_number": { "masked": false }
  },
  "face_match": {
    "selfie_image_score": 0.82,
    "id_photo_score": 0.44,
    "similarity_score": 0.39
  },
  "forgery_checks": {
    "template_match": false,
    "glare_detected": true,
    "tampering_signals": ["edge_noise", "misalignment"]
  },
  "compliance_checks": {
    "age_check": "FAILED",
    "aadhaar_masking": "FAILED",
    "face_match_threshold": "FAILED"
  },
  "integrity_issues": ["partial crop", "low resolution"],
  "language_hint": "en"
}

return something like:  

Your Aadhaar verification failed due to multiple reasons:
- The Aadhaar number appears unmasked, which violates compliance norms.
- The system couldn't confirm that the selfie matches the photo on the Aadhaar card.
- The document seems to be partially cropped and of low quality.
- You must be 18 years or older for verification, but your age seems to be under 18.
Please upload a clear, unedited Aadhaar image with proper masking, and ensure a well-lit selfie with your face clearly visible.

"""

In [ ]:

user_prompt = f"""
Below is a structured JSON result from Digio's Full IDV pipeline.

Please analyze it and generate a user-friendly explanation:
- State if verification passed or failed.
- If failed, explain all the reasons clearly.
- Suggest what the user should do to fix each issue.

JSON Input:
```json
{json.dumps(idv_json, indent=2)} ``` """

#### guided by [[link]](https://platform.openai.com/docs/guides/responses-vs-chat-completions?api-mode=responses)

In [27]:
def query_llm(system_prompt, user_prompt, model="gpt-4o"):
    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0.3
    )
    return response.choices[0].message.content


In [ ]:
result = query_llm(system_prompt,user_prompt)
print(result)